[View in Colaboratory](https://colab.research.google.com/github/Jacobh2/colabflow/blob/master/Colabflow.ipynb)

In [1]:
print("Hello world! again")

Hello world! again


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import gc
from AMSGrad import AMSGrad
tf.logging.set_verbosity(tf.logging.INFO)

from tqdm import tqdm
from tqdm import trange

from os import mkdir
from os.path import abspath

from random import choice
from random import seed

#from official.mnist import dataset as mnist
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./MNIST_data/")

tfd = tf.contrib.distributions

EPOCH = 20
BATCH_SIZE = 100
SAMPLE_SIZE = 10
STD_SAMPLE_RANGE=5
LATENT_SPACE_STANDARD=10
LEARNING_RATE=0.0005
BETA_DISENTANGLE=100          #Set to 1 to "turn off"
CAPACITY_CONTROL_MAX=15     #Set to 0 to "turn off"


def make_prior(code_size=LATENT_SPACE_STANDARD):
    with tf.name_scope('prior'):
        mean = tf.zeros([code_size])
        stddev = tf.ones([code_size])
        return tfd.MultivariateNormalDiag(mean, stddev)


def make_encoder(images, code_size=LATENT_SPACE_STANDARD):
    with tf.name_scope('encoder'):
        images = tf.layers.flatten(images)
        hidden = tf.layers.dense(images, 200, tf.nn.relu)
        hidden = tf.layers.dense(hidden, 200, tf.nn.relu)
        mean = tf.layers.dense(hidden, code_size, name='encoder_mean')
        stddev = tf.layers.dense(hidden, code_size, tf.nn.softplus, name='encoder_stddev')
        return tfd.MultivariateNormalDiag(mean, stddev, name='mycoolname')


def make_decoder(code, data_shape=[28, 28]):
    with tf.name_scope('decoder'):
        hidden = tf.layers.dense(code, 200, tf.nn.relu)
        hidden = tf.layers.dense(hidden, 200, tf.nn.relu)
        logit = tf.layers.dense(hidden, np.prod(data_shape))
        logit = tf.reshape(logit, [-1] + data_shape)
        return tfd.Independent(tfd.Bernoulli(logit), len(data_shape))

def create_sample_codes_line(latent_dimentions, codes, beta, samples_per_dimention=20):
    MIN = -STD_SAMPLE_RANGE
    MAX = STD_SAMPLE_RANGE

    #create coordinates holder
    # coordinates = np.zeros((samples_per_dimention*latent_dimentions,latent_dimentions))

    #coordinates = codes[:samples_per_dimention*latent_dimentions]
    # print(codes[0].shape)
    # print(np.mean(codes[0]))
    # print(np.std(codes[0]))
    seed(beta)
    selected_image_from_batch = choice(codes)

    coordinates = np.repeat([selected_image_from_batch],samples_per_dimention*latent_dimentions, axis=0)
    #.reshape([samples_per_dimention*latent_dimentions,latent_dimentions])

    #fill coordinates with linspace leaving all other values in same coordinate as zero
    for i in range(latent_dimentions):
        linspace_min = MIN + selected_image_from_batch[i]
        linspace_max = MAX + selected_image_from_batch[i]

        linspace = np.linspace(linspace_min, linspace_max, samples_per_dimention)


        coordinates[i*samples_per_dimention:(i+1)*samples_per_dimention, i] = linspace
        # Original:
        #coordinates[i*samples_per_dimention:(i+1)*samples_per_dimention,i] = np.linspace(MIN, MAX, samples_per_dimention)

        # New test:


    return coordinates

def create_sample_codes_mesh(latent_dimentions=2, samples_per_dimention=20):
    MIN = -STD_SAMPLE_RANGE
    MAX = STD_SAMPLE_RANGE

    #create values for coordinates seperated on x values and y values
    x,y=np.meshgrid(np.linspace(MIN, MAX, samples_per_dimention),np.linspace(MIN, MAX, samples_per_dimention))

    #merge x and y-values to coordinates
    coordinates=np.array(list(zip(x.ravel(),y.ravel())))
    
    return coordinates

def plot_samples_batch(session, ax, coordinates, latent_dimentions=LATENT_SPACE_STANDARD, samples_per_dimention=20):
    
    #print("RUNNING IMAGES")
    # Run the values
    all_images = session.run(decoder_output_reshaped, feed_dict={
        decoder_input: coordinates
    })
    
    #Prepare to order pictures by importance of neuron (i.e. importance for change of pixel)
    order_value=list()
    for i in range(latent_dimentions):
        order_value.append((np.mean(np.square((all_images[i*samples_per_dimention]-all_images[(i+1)*samples_per_dimention-1]))),i))
    #Aaaaaand the ordering itself
    order_value.sort(reverse=True)
    index_sorted = [x for _, x in order_value]

    #print("PLOTTING")
    
    for ax_index_x in range(latent_dimentions):
        ax_x = ax[ax_index_x]
        for ax_index_y in range(samples_per_dimention):
            ax_x_y = ax_x[ax_index_y]
            ax_x_y.imshow(all_images[index_sorted[ax_index_x]*samples_per_dimention+ax_index_y], cmap='gray')
            ax_x_y.axis('off')

# Skapa variant av input med annan shape
images = tf.placeholder(tf.float32, [None, 28, 28])

decoder_input = tf.placeholder(tf.float32, [None, LATENT_SPACE_STANDARD])

capacity_control = tf.placeholder(tf.float32, ())
beta_disentangle_var = tf.placeholder(tf.float32, ())

# make_encoder = tf.make_template('encoder', make_encoder)
# make_decoder = tf.make_template('decoder', make_decoder)

# Gör vår 'prior' (att sample logit från)
prior = make_prior()

# encode:a input till en 'posterior'
posterior = make_encoder(images)

posterior_mean = posterior.mean()
posterior_stddev = posterior.stddev()

with tf.variable_scope("model", reuse=False):
    # Decode:a latent z men independent
    code = posterior.sample()
    dist = make_decoder(code)

with tf.variable_scope("model", reuse=True):
    # Decode:a logit som vi samplar från och ta dess mean
    sample = make_decoder(prior.sample(10)).mean() # for visualization

with tf.variable_scope("model", reuse=True):
    # Decode:a logit som vi samplar från och ta dess mean
    decoder_output = make_decoder(decoder_input).mean() # for visualization
    decoder_output_reshaped = tf.reshape(decoder_output, [-1, 28, 28])

# Evidence Lower BOund (cost-fn)
elbo = tf.reduce_mean(dist.log_prob(images) - beta_disentangle_var * tf.abs(tfd.kl_divergence(posterior, prior)-capacity_control))

# Skapa optimerare (Adam) och minimera 
#optimize = tf.train.AdamOptimizer(LEARNING_RATE).minimize(-elbo)
optimize = AMSGrad(learning_rate=LEARNING_RATE, beta1=0.9, beta2=0.99, epsilon=1e-8).minimize(-elbo)

training_data = mnist.train
test_data = mnist.test

def run(filename='./ascoolt3.png', beta=BETA_DISENTANGLE, t=None):

    try:
        asb_path = '/'.join(abspath(filename).split('/')[:-1])
        mkdir(asb_path)
    except Exception:
        pass

    latent_dimentions=LATENT_SPACE_STANDARD
    samples_per_dimention=20

    with tf.Session() as session:
        session.run(tf.global_variables_initializer())
        test_elbo=None
        for epoch in range(EPOCH):
            
            if t:
                t.set_postfix(epoch=epoch, beta=beta)

            test_elbo,test_codes=session.run([elbo,code], feed_dict={
                    images: test_data.images.reshape([-1, 28, 28]), 
                    capacity_control: (CAPACITY_CONTROL_MAX*(epoch+1)/EPOCH),
                    beta_disentangle_var: beta})
            #print(elbo)
            for train_step in range(training_data.num_examples//BATCH_SIZE):
                image_data, label = training_data.next_batch(BATCH_SIZE)
                session.run(optimize, feed_dict={
                    images: image_data.reshape([-1, 28, 28]), 
                    capacity_control: (CAPACITY_CONTROL_MAX*(epoch+1)/EPOCH),
                    beta_disentangle_var: beta})

        coordinates = create_sample_codes_line(latent_dimentions,test_codes, beta)
        fig, ax = plt.subplots(nrows=int(coordinates.shape[0]/samples_per_dimention), ncols=samples_per_dimention,
                figsize=((samples_per_dimention, int(coordinates.shape[0]/samples_per_dimention))))
        plot_samples_batch(session, ax, coordinates)
    
    plt.savefig(filename, dpi=300, transparent=True, bbox_inches='tight')
    #print("Picture saved!")
    #plt.show()
        # for i in range(SAMPLE_SIZE):
        #     image = tf.reshape(sample[i],[28,28])
        #     image = session.run(image)
        #     plt.imshow(image, cmap='gray')  
        #     plt.show()

def gridsearch_beta():
    #With capacity control
    for i in tqdm(range(0,100,10), desc="Total process", unit="configs"):
        run('./beta/' + str(i) + 'capcont.png', beta=i)
    #Without capacity control
    global CAPACITY_CONTROL_MAX
    CAPACITY_CONTROL_MAX=0
    for i in tqdm(range(0,100,10), desc="Total process", unit="configs"):
        run(filename='./beta/' + str(i) + '.png', beta=i)
        gc.collect()


def random_search_beta(MIN=15,MAX=50,INSTANCES=5):
    search_points=list(np.random.choice(range(MIN,MAX),INSTANCES,replace=False))
    print("Söker av beta =", search_points)
    #With capacity control
    with trange(len(search_points), desc='Total Networks', unit='Configs') as t:
        for ii in t:
            i = search_points[ii]
            t.set_description('BETA %i' % i)
            # for i in tqdm(search_points, desc="Total process", unit="configs"):
            run('./beta/' + str(i) + 'capcont.png', beta=i, t=t)
    #Without capacity control
    global CAPACITY_CONTROL_MAX
    CAPACITY_CONTROL_MAX=0
    with trange(len(search_points), desc='Total Networks', unit='Configs') as t:
        for ii in t:
            i = search_points[ii]
            t.set_description('BETA %i' % i)
            # for i in tqdm(search_points, desc="Total process", unit="configs"):
            run(filename='./beta/' + str(i) + '.png', beta=i, t=t)
            # gc.collect()

if __name__ == '__main__':
    # print("A", dir(posterior))
    # print("B", posterior.mean())
    # print("C", posterior.stddev())
    # print("TEST IMAGES:", mnist.test.labels)
    #run()
    #gridsearch_beta()
    random_search_beta()
    # for i in tf.trainable_variables():
    #     print("VAR:", i)


ModuleNotFoundError: ignored